In [2]:
# ! pip install python-dotenv

In [6]:
from dotenv import load_dotenv
import os

load_dotenv()

# Loading Credentials
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("API_VERSION")
api_endpoint = os.getenv("AZURE_BASE_URL")
together_api = os.getenv("TOGETHER_API_KEY")
chat_model = os.getenv("CHAT_MODEL")
embedding_model = os.getenv("EMBEDDING_MODEL")

In [7]:
# ! pip install openai
# ! pip install langchain langchain_community

### Langchain model initialization

In [8]:
from langchain.chat_models import AzureChatOpenAI


llm = AzureChatOpenAI(
    api_key=api_key, 
    model_name=chat_model,
    api_version = api_version,
    azure_endpoint = api_endpoint,
    temperature=0
    )

/tmp/ipykernel_4890/1300644453.py:4: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(


In [29]:
''' 
Prompt Creation
'''

from langchain.prompts import PromptTemplate

# Define a prompt template that instructs the model to analyze and expand on the topic
research_prompt = PromptTemplate(
    input_variables=["topic"],
    template=(
        '''
        You are a research assistant. The user has asked the following research question or provided this topic: '{topic}'.
        Please analyze the topic, explain it back to the user, and suggest 4 additional related questions and ideas that could help expand the research. 
        Generate a list of 5 search queries (including the original one) or topics that could be helpful in researching about the topic.
        ''' 
    )
)


In [30]:
from langchain import LLMChain
from langchain.chains import SimpleSequentialChain

# Create an LLMChain using the defined prompt template
research_chain = LLMChain(
    llm=llm,
    prompt=research_prompt
)

# Simple Sequential Chain to handle the processing
sequential_chain = SimpleSequentialChain(
    chains=[research_chain],
    verbose=True
)

/tmp/ipykernel_4890/3597435984.py:5: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  research_chain = LLMChain(


In [11]:
topic = "The impact of climate change on polar bear populations"

In [ ]:
''' 
Sample Generation : Testing our initializations
'''

def generate_research_insights(topic):
    # Run the sequential chain with the provided topic
    response = sequential_chain.run(topic)
    return response

# Example usage
result = generate_research_insights(topic)
print(result)

/tmp/ipykernel_27505/4203453993.py:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = sequential_chain.run(topic)




> Entering new SimpleSequentialChain chain...
### Topic Analysis

The topic "The impact of climate change on polar bear populations" focuses on understanding how the changing climate, particularly in the Arctic regions, affects polar bears, their habitats, and their overall survival. Climate change leads to rising temperatures, melting ice caps, and altered ecosystems, which directly impact polar bears, as they rely on sea ice for hunting seals, their primary food source. As the ice diminishes, polar bears may face challenges such as reduced access to food, increased competition, and changes in their breeding patterns. This topic is crucial for conservation efforts and understanding the broader implications of climate change on biodiversity.

### Related Questions and Ideas

1. **How does the melting Arctic ice affect the hunting behavior and diet of polar bears?**
   - Investigating the specific changes in hunting patterns and dietary shifts due to ice loss.

2. **What are the long-

### We'll be using below resources to gather more information around our reserrch
1. YouTube
2. Semantic Scholar
3. Web Articles and Blogs

#### YouTube

In [21]:
# ! pip install --upgrade youtube_search

In [22]:
from langchain_community.tools import YouTubeSearchTool # this library has Python function for searching for youtube videosss

tool = YouTubeSearchTool()
tool.run(topic, 5) # 5 is the number we want the tool to return us from the avaiable videos

['https://www.youtube.com/watch?v=0TXbUt1j6Mc&pp=ygU2VGhlIGltcGFjdCBvZiBjbGltYXRlIGNoYW5nZSBvbiBwb2xhciBiZWFyIHBvcHVsYXRpb25z0gcJCU8JAYcqIYzv', 'https://www.youtube.com/watch?v=V-YFOlwwe_g&pp=ygU2VGhlIGltcGFjdCBvZiBjbGltYXRlIGNoYW5nZSBvbiBwb2xhciBiZWFyIHBvcHVsYXRpb25z']

"['https://www.youtube.com/watch?v=0TXbUt1j6Mc&pp=ygU2VGhlIGltcGFjdCBvZiBjbGltYXRlIGNoYW5nZSBvbiBwb2xhciBiZWFyIHBvcHVsYXRpb25z0gcJCU8JAYcqIYzv', 'https://www.youtube.com/watch?v=V-YFOlwwe_g&pp=ygU2VGhlIGltcGFjdCBvZiBjbGltYXRlIGNoYW5nZSBvbiBwb2xhciBiZWFyIHBvcHVsYXRpb25z']"

#### Wikipedia

In [24]:
# ! pip install wikipedia
# ! pip install xmltodict # formating the output

In [27]:
''' 
Using only Wikipedia retriever
'''
from langchain_community.retrievers import WikipediaRetriever

# retriever cell
retriever = WikipediaRetriever()
retriever.invoke(topic)

[Document(metadata={'title': 'Polar bear', 'summary': "The polar bear (Ursus maritimus) is a large bear native to the Arctic and nearby areas. It is closely related to the brown bear, and the two species can interbreed. The polar bear is the largest extant species of bear and land carnivore, with adult males weighing 300–800 kg (660–1,760 lb). The species is sexually dimorphic, as adult females are much smaller. The polar bear is white- or yellowish-furred with black skin and a thick layer of fat. It is more slender than the brown bear, with a narrower skull, longer neck and lower shoulder hump. Its teeth are sharper and more adapted to cutting meat. The paws are large and allow the bear to walk on ice and paddle in the water.\nPolar bears are both terrestrial and pagophilic (ice-living) and are considered marine mammals because of their dependence on marine ecosystems. They prefer the annual sea ice but live on land when the ice melts in the summer. They are mostly carnivorous and spe

In [ ]:
''' 
Using Agents to retrieve relevant information from Wikipedia
'''
from langchain.agents import AgentType, initialize_agent
from langchain.agents import load_tools

# Wikipedia tool agent
tools = load_tools(
                    ["wikipedia"], 
                    llm=llm
                    )

agent= initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

agent(topic)

#### Semantic Scholar
Semantic Scholar uses natural language and machine learning to find relevant results across all disciplines, including biology, medicine, computer science, geography, business, history, and economics. It includes a scientific literature graph and billions of citations, and its corpus contains nearly 200 million academic papers from publisher partnerships, data providers, and web crawls.

In [ ]:
# ! pip install --upgrade --quiet  semanticscholar

In [38]:
from langchain_community.tools.semanticscholar.tool import SemanticScholarQueryRun
from langchain_community.utilities.semanticscholar import SemanticScholarAPIWrapper

api_wrapper = SemanticScholarAPIWrapper(doc_content_chars_max=1000, top_k_results=5)
tools = [SemanticScholarQueryRun(api_wrapper=api_wrapper)]

In [45]:
''' 
Here for Semantic Scholar we're creating a Prompt to retrive more contectual information. In above case for Wikipedia we've not created any prompt if you check in tools.
'''

from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert researcher.",
        ),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)


agent_executor = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_MULTI_FUNCTIONS,
    prompt=prompt,
    verbose=True
)

In [46]:
response = agent_executor.invoke(
    {
        "input": f"""query: {topic}, for each of the relevant paper that you find :Create 3 different sections: List of papers with url for it, summary of that paper and citations in that paper, do this for each paper   """
    }
)



> Entering new AgentExecutor chain...

Invoking: `semanticscholar` with `{'query': 'The impact of climate change on polar bear populations'}`


Published year: 2016
Title: Modelling the Impact of Climate Change on the Polar Bear Population in Western Hudson Bay
Authors: Nicole Bastow
Abstract: None

Invoking: `semanticscholar` with `{'query': 'The impact of climate change on polar bear populations'}`


Published year: 2016
Title: Modelling the Impact of Climate Change on the Polar Bear Population in Western Hudson Bay
Authors: Nicole Bastow
Abstract: None
I found a relevant paper on the impact of climate change on polar bear populations. Here are the details:

### 1. List of Papers
- **Title**: Modelling the Impact of Climate Change on the Polar Bear Population in Western Hudson Bay
- **Authors**: Nicole Bastow
- **Published Year**: 2016
- **URL**: [Link to Paper](https://www.semanticscholar.org/paper/Modelling-the-Impact-of-Climate-Change-on-the-Bastow/)

### 2. Summary of the Paper

#### Playwright Browser

- Web Scraping: Automatically extract data from web pages, such as product prices, reviews, or contact information.
- Web Testing: Automate testing of web applications by simulating user interactions and verifying expected outcomes.
- Research: Gather information from various websites on a specific topic and summarize the findings.
- Personal Automation: Automate tasks that involve interacting with web applications, such as checking email, managing social media, or booking appointments.

In [ ]:
''' 
Install below using Terminal
'''

# ! pip install playwright beautifulsoup4
# ! playwright install
# sudo apt-get install libgstreamer-plugins-bad1.0-0 libavif16

' \nInstall below using Terminal\n'

In [1]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit

from langchain_community.tools.playwright.utils import (
    create_async_playwright_browser,  # A synchronous browser is available, though it isn't compatible with jupyter.\n",      },
)
import nest_asyncio

nest_asyncio.apply()

In [2]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import (
    create_async_playwright_browser,  # A synchronous browser is available, though it isn't compatible with jupyter.\n",	  },
)
# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio
nest_asyncio.apply()

In [3]:
async_browser = create_async_playwright_browser()
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()
tools

[ClickTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/home/varun/.cache/ms-playwright/chromium-1161/chrome-linux/chrome> version=134.0.6998.35>),
 NavigateTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/home/varun/.cache/ms-playwright/chromium-1161/chrome-linux/chrome> version=134.0.6998.35>),
 NavigateBackTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/home/varun/.cache/ms-playwright/chromium-1161/chrome-linux/chrome> version=134.0.6998.35>),
 ExtractTextTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/home/varun/.cache/ms-playwright/chromium-1161/chrome-linux/chrome> version=134.0.6998.35>),
 ExtractHyperlinksTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/home/varun/.cache/ms-playwright/chromium-1161/chrome-linux/chrome> version=134.0.6998.35>),
 GetElementsTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/home/var

In [9]:
from langchain.agents import AgentType, initialize_agent

agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

/tmp/ipykernel_4890/1026851030.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_chain = initialize_agent(


In [12]:
result = await agent_chain.arun(f"""find 5 artiles on "{topic}" and return the links for those articles and blogs
You dont have to return all the links or urls just get the top 5 urls other than google urls and return it with article name. dont go in as loop of finding and filtering urls """)
print(result)

/tmp/ipykernel_4890/521848347.py:1: LangChainDeprecationWarning: The method `Chain.arun` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~ainvoke` instead.
  result = await agent_chain.arun(f"""find 5 artiles on "{topic}" and return the links for those articles and blogs




> Entering new AgentExecutor chain...
Thought: I need to search for articles related to "The impact of climate change on polar bear populations" and extract the top 5 relevant links without including Google URLs. I will navigate to a search engine and perform the search.

Action:
```json
{
  "action": "navigate_browser",
  "action_input": "https://www.bing.com/search?q=The+impact+of+climate+change+on+polar+bear+populations"
}
```

Observation: Navigating to https://www.bing.com/search?q=The+impact+of+climate+change+on+polar+bear+populations returned status code 200
Thought:I need to extract the hyperlinks from the search results page to find articles related to "The impact of climate change on polar bear populations."

Action:
```json
{
  "action": "extract_hyperlinks",
  "action_input": {"absolute_urls": true}
}
```


Observation: ["https://www.bing.com/search?q=The+impact+of+climate+change+on+polar+bear+populations&rdr=1&rdrig=C876BF055D7D453CB0A6A2189D4E058B&setlang=ur&sid=2866C65

#### PubMed
* PubMed is a free, searchable database of citations and abstracts for biomedical and life sciences literature created by the National Library of Medicine (NLM). It contains over 37 million citations and abstracts from more than 5,600 journals in 30 languages, dating back to 1946. PubMed covers journal articles in medicine, nursing, dentistry, veterinary medicine, and the health care system.

In [14]:
from langchain_community.retrievers import PubMedRetriever
retriever = PubMedRetriever()

In [18]:
retriever.invoke(topic)

[Document(metadata={'uid': '39929952', 'Title': 'Model sensitivity limits attribution of greenhouse gas emissions to polar bear demographic rates.', 'Published': '2025-02-10', 'Copyright Information': '© 2025. This is a U.S. Government work and not under copyright protection in the US; foreign copyright protection may apply.'}, page_content='Greenhouse gas emissions continue to increase and negatively affect sea ice conditions that polar bears rely on. It is therefore important to better understand how specific emissions levels affect polar bear demography. A recent study proposed a framework to address this issue, but sensitivity to decisions rules of the approach may limit its utility. We tested how sensitive the approach is to decisions rules related to sea ice concentration, choice of subpopulation boundaries, and modeling choices for bears in the Chukchi Sea and Southern Beaufort Sea subpopulations. We found that the number of ice-free days, number of fasting days, and when 10% of

In [19]:
from langchain_community.tools.pubmed.tool import PubmedQueryRun
tool = PubmedQueryRun()

tool.invoke(topic)

"Published: 2025-02-10\nTitle: Model sensitivity limits attribution of greenhouse gas emissions to polar bear demographic rates.\nCopyright Information: © 2025. This is a U.S. Government work and not under copyright protection in the US; foreign copyright protection may apply.\nSummary::\nGreenhouse gas emissions continue to increase and negatively affect sea ice conditions that polar bears rely on. It is therefore important to better understand how specific emissions levels affect polar bear demography. A recent study proposed a framework to address this issue, but sensitivity to decisions rules of the approach may limit its utility. We tested how sensitive the approach is to decisions rules related to sea ice concentration, choice of subpopulation boundaries, and modeling choices for bears in the Chukchi Sea and Southern Beaufort Sea subpopulations. We found that the number of ice-free days, number of fasting days, and when 10% of reproductive females exhibited recruitment failure va

In [23]:
tools = load_tools(["pubmed"], llm=llm)
agent= initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

# agent(topic)

#### Human as a Tool
* Human are AGI so they can certainly be used as a tool to help out AI agent when it is confused, or even for approvals sometimes.

* The agent can ben armed with an extensive array of tools to use in order to answer the question. And one of those tools, or actions can be a Human, hence the agent follows a Human-In-The-Loop approach or as some referred to it, Human as a Tool.

In [24]:
# from langchain.agents import AgentType, initialize_agent, load_tools

#Simple function for looped inpt
def get_input() -> str:
    print("Insert your text. Enter 'q' to end.")
    contents = []
    while True:
        try:
            line = input()
        except EOFError:
            break
        if line == "q":
            break
        contents.append(line)
    return "\n".join(contents)

human_tool = load_tools(["human"], llm=llm, input_func=get_input)

In [25]:
human_agent_chain = initialize_agent(
    human_tool,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

In [26]:
human_agent_chain.run("I need help finding who said the quote. just reply with the name of the person who said the quote .")

/tmp/ipykernel_4890/4281521762.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  human_agent_chain.run("I need help finding who said the quote. just reply with the name of the person who said the quote .")




> Entering new AgentExecutor chain...
I need to know the specific quote in question to identify the person who said it. 
Action: human
Action Input: Can you provide the quote you need help with? 

Can you provide the quote you need help with?
Insert your text. Enter 'q' to end.

Observation: sare jga se acha hindustan hmara
Thought:The quote "sare jga se acha hindustan hmara" translates to "Our India is better than all other places." This quote is attributed to the famous Indian poet and philosopher, **Allama Iqbal**. 

Final Answer: Allama Iqbal

> Finished chain.


'Allama Iqbal'

### Building out final Agent

In [2]:

human_ideas=""
def generate_research_insights_with_human_input(topic):
    # Step 1: Get the initial research ideas and queries from the LLM
    llm_response = sequential_chain.run(topic)
    print("LLM Response:\n", llm_response)

    # Step 2: Ask the user for additional input
    human_ideas = human_agent_chain.run(human_agent_prompt)
    print("Human Response:\n", human_ideas)

    # Combine LLM output with human input
    combined_response = f"{llm_response}\n\nAdditional Ideas and Queries from Human Input:\n{human_ideas}"

    return combined_response

# Example usage
topic = "The impact of climate change on polar bear populations"
human_agent_prompt="""your only job is to take user input and clean it, sanitize it for spelling mistakes , remove special characters and return the inputs.
 you dont need to do anything else. just return the cleaned output in one step. you just have to take the user input once not more than that"""

# result = generate_research_insights_with_human_input(topic)
# print(result)

-  instead of apending the input you can pass it back to the llm for more refinement in the ideas